In [248]:
import numpy as np
from itertools import chain, combinations
from scipy import sparse

In [249]:
# configuration model
# https://en.wikipedia.org/wiki/Configuration_model

n = 16
deg_c = 4 # w_r. Every check has this many bits in it
deg_v = 3 # w_c. Every bit is in this many checks
num_checks = (n*deg_v)//deg_c
k = n - num_checks

vs = np.array([[j for i in range(deg_v)] for j in range(n)]).flatten()
cs = np.array([[j for i in range(deg_c)] for j in range(num_checks)]).flatten()

H = np.zeros((num_checks, n), dtype=bool)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            vs = np.delete(vs, v_ind)
            cs =np.delete(cs, c_ind)

H = sparse.csc_matrix(H)

In [250]:
H = np.loadtxt('./good_ldpc_codes/16_4_3.txt', dtype=bool)

In [251]:
hx1 = sparse.kron(H, np.eye(H.shape[1], dtype=bool))
hx2 = sparse.kron(np.eye(H.shape[0], dtype=bool), H.T)
Hx = sparse.csr_matrix(sparse.hstack([hx1, hx2], ))

hz1 = sparse.kron(np.eye(H.shape[1], dtype=bool), H)
hz2 = sparse.kron(H.T, np.eye(H.shape[0], dtype=bool))
Hz = sparse.csr_matrix(sparse.hstack([hz1, hz2]))

In [252]:
2**Hx[0].nnz*Hx.shape[0] # number of error syndromes we have to check

24576

In [ ]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

eF = np.zeros(Hx.shape[1], dtype=bool)
def syn_from_F(F, H):
    eF_cp = eF.copy()
    np.put(eF_cp, F, [1])
    return set(np.where(H @ eF_cp)[0])

Fx = [list(powerset(Hx[i].indices))[1:] for i in range(Hx.shape[0])]
Fx = list(chain(*Fx)) # can take set of this list for slight reduction in size
Fx_coords = np.array([(item, i) for i, j in enumerate(Fx) for item in j])
Fx_arr = sparse.coo_matrix(([True for i in range(len(Fx_coords))], (Fx_coords[:,0], Fx_coords[:,1])), dtype=bool).tocsc()
sigma_Fx = (Hz.tocsc() @ Fx_arr).transpose()

# Fx = [set(g) for g in Fx] 
Fz = [list(powerset(Hz[i].indices))[1:] for i in range(Hz.shape[0])]
Fz = list(chain(*Fz))
Fz_coords = np.array([(item, i) for i, j in enumerate(Fz) for item in j])
Fz_arr = sparse.coo_matrix(([True for i in range(len(Fz_coords))], (Fz_coords[:,0], Fz_coords[:,1])), dtype=bool).tocsc()
sigma_Fz = (Hx.tocsc() @ Fz_arr).transpose()

# Fz = [set(g) for g in Fz]

# sigma_Fx = [syn_from_F(g, Hz) for g in Fx] # set of indices where syndrome is 1
# sigma_Fz = [syn_from_F(g, Hx) for g in Fz]

In [8]:
def ssf(syn, F, sigma_F):
    # given a syndrome, syn: sigma_x or sigma_z 
    # x_z false for x stabilizers, true for z stabilizers
    s = set(np.where(syn.copy())[0])
    e = set()
    
    zipped = zip(F, sigma_F)

    while 1:
        max = -1
        max_gen = None
        max_sigma_gen = None
        
        for g, sigma_g in zipped:
            s_i = s ^ sigma_g
            len_s = len(s)
            len_s_i = len(s_i)
            if (len_s_i < len_s):
                rel_weight = (len_s - len_s_i) / len(g)
                if (rel_weight > max):
                    max = rel_weight
                    max_gen = g
                    max_sigma_gen = sigma_g

        if (max == -1):
            if (len(s) == 0):
                return e
            else:
                return s
        else:
            e = e ^ set(max_gen)
            s = s ^ max_sigma_gen

In [228]:
p = 0.005

# how is this able to decode an error with more than (d-1)/2 errors?

for i in range(1):
    eX = [True if np.random.uniform() < p else False for i in range(Hx.shape[1])]
    eZ = [True if np.random.uniform() < p else False for i in range(Hz.shape[1])]

    sigma_eX = Hx.dot(eX)
    # s = set(np.where(sigma_eX.copy())[0])
    # sigma_eZ = np.dot(Hz, eZ)
    # e1 = ssf(sigma_eX, Fz, sigma_Fz)
    # e2 = ssf(sigma_eZ, True)
    # print(e1, np.where(eX))
    # new_e = e1 ^ set(np.where(eX)[0])
    # print(len(np.where(eX)[0]), len(np.where(eZ)[0]))

In [230]:
sigma_eX ^ sigma_Fx

TypeError: unsupported operand type(s) for ^: 'numpy.ndarray' and 'csc_matrix'

In [279]:
zipped =zip(Fx, sigma_Fx)
s = set(np.where(sigma_eX.copy())[0])
def test2():
    for g, sigma_g in zipped:
        s_i = s ^ sigma_g
        len_s_i = len(s_i)
    # pass
        # len_s = len(s)
        # len_s_i = len(s_i)
        # if (len_s_i < len_s):
        #     rel_weight = (len_s - len_s_i) / len(g)
        #     if (rel_weight > max):
        #         max = rel_weight
        #         max_gen = g
        #         max_sigma_gen = sigma_g

In [10]:
%timeit ssf(sigma_eX, Fz, sigma_Fz)

23.3 ms ± 697 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [338]:
%timeit s ^ sigma_Fx[0]

424 ns ± 40.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [469]:


def test():
    s = 0
    for i in range(6000000):
        s += i
test()

In [470]:
%timeit test()

541 ms ± 62.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
